# NB16: Higher-order functions

## Programming Fundamentals

## L.EIC/2022-23

#### Nuno Macedo$^{1}$, João Correia Lopes$^{1}$, Pedro Vasconcelos$^{2}$
$^{1}$FEUP/DEI & INESC TEC\
$^{2}$FCUP/DCC & LIACC

> “The only way to learn a new programming language is by writing programs in it”

Dennis Ritchie

## Goals

By the end of this class, the student should be able to:

- Describe the use higher-order functions
- Describe the use nested functions and closures
- Describe the use function *currying* and *uncurrying*
- Describe the use of partial function application
- Describe the use the operators available in Python

## Bibliography

- David Mertz, *Functional Programming in Python*, O'Reilly Media, 2015
[[HTML]](https://www.oreilly.com/library/view/functional-programming-in/9781492048633/)
- Composing Programs, a free online introduction to programming and computer science (Section 1.6)
[[HTML]](https://composingprograms.com/pages/16-higher-order-functions.html)


# 16 Higher-order functions (HOF)

## 16.1 Introduction

- A **higher-order function** is a function that
     - takes another function as *argument*; or
     - returns a function as *result*

- We've already seen some higher-order functions
   - `map`, `filter` and `reduce` take a function as first argument;
   - `sort` and `sorted` take a optional function for comparing values

- HOFs are useful because they allow us to *abstract* behaviour using
  functions
- This allows building "generic" functions that express complex behaviour
  without repetition (see the [DRY principle](https://en.wikipedia.org/wiki/Don%27t_repeat_yourself))

### Sorting: An Example of HOF

- In order to define non-default sorting in Python, both the `sorted()` function and the list’s `sort()` method accept a key argument

- The value passed to this argument needs to be a function object that returns the sorting key for any item in the list or iterable

```
>>> def second_element(t):
...     return t[1]
...
>>> ledzep = [('Guitar', 'Jimmy'), ('Vocals', 'Robert'), ('Bass', 'John Paul'), ('Drums', 'John')]

>>> sorted(ledzep)
[('Bass', 'John Paul'), ('Drums', 'John'), ('Guitar', 'Jimmy'), ('Vocals', 'Robert')]

>>> sorted(ledzep, key=second_element)
[('Guitar', 'Jimmy'), ('Drums', 'John'), ('Bass', 'John Paul'), ('Vocals', 'Robert')]
```

## 16.2 First-class functions

> When you say that a language has first-class functions, it means that the language treats functions as values:

- You can store the function in a variable
- You can pass the function as a parameter to another function
- You can return the function from another function
- You can store them in data structures such as dictionaries, lists, etc.
- In Python a function is an instance of the Object type

### Functions as objects

A Python program to illustrate functions can be treated as objects:

In [ ]:
def greet(name):
    return "Hello, " + name + "."
def goodbye(name):
    return "Goodbye " + name + "!"

f = greet
print(f("Alex"))
g = goodbye
print(g("Alex"))


In [ ]:
actions = [greet, goodbye, greet, goodbye]
names = ["Alex", "Alex", "Eve", "Eve"]
for (action, name) in zip(actions,names):
     print(action(name))


## 16.3 First-class vs Higher-order functions

### Higher-order functions

- To express certain general patterns as named concepts, we will need to construct functions that can accept other functions as arguments or return functions as values

- Functions that manipulate functions are called **higher-order functions**

- higher-order functions (HOF) can serve as powerful abstraction mechanisms, vastly increasing the expressive power of our language

### “higher-order” vs. “first-class”

- The “higher-order” (HOF) concept can be applied to functions in general, like functions in the mathematical sense

- The “first-class” concept only has to do with functions in programming languages:

  - It’s not used when referring to a function, such as “a first-class function”

  - It’s much more common to say that “a language has/hasn’t first-class function support”

- The two things are closely related, as it’s hard to imagine a language with first-class functions that would not also support higher-order functions, and conversely a language with higher-order functions but without first-class function support.

## 16.4 Functions as parameters

### Example: summing squares and cubes

Consider the functions `sum_squares()`, which computes the sum of the squares of natural numbers up to `n`

In [ ]:
def sum_squares(n):
    total = 0
    k = 1
    while k <= n:
        total = total + k*k
        k = k + 1
    return total

print(sum_squares(100))

###

Consider now a similar the functions `sum_cubes()`, which computes the sum of the cubes of natural numbers up to `n`

In [ ]:
def sum_cubes(n):
    total = 0
    k = 1
    while k <= n:
        total = total + k*k*k
        k = k + 1
    return total

print(sum_cubes(100))

* Functions `sum_squares` and `sum_cubes` clearly share a common underlying pattern

* They are for the most part identical, differing only in name and the **function of k used to compute the term** to be added

### Abstraction, first attempt

* Let us  define a more general function `summation` that generalizes the two functions above
* `summation` takes two arguments:
   - the upper bound `n`
   - a function `term` that computes the kth term



In [ ]:
def summation(n, term):
    total = 0
    k = 1
    while k <= n:
        total = total + term(k)
        k = k + 1
    return total


* Now we can re-write the two original function as special cases of `summation`

* This avoids the code duplication in original definitions

In [ ]:

def sum_squares(n):
    return summation(n, lambda x: x*x)

def sum_cubes(n):
    return summation(n, lambda x: x*x*x)

print(sum_squares(100))
print(sum_cubes(100))

See how this works in [Python tutor](http://www.pythontutor.com/visualize.html#mode=edit)

### Summing naturals

To sum the natural numbers we can pass the identity function to `summation`.

In [ ]:
def sum_naturals(n):
    return summation(n, lambda x: x)

print(sum_naturals(100))

### Abstraction, second attempt

* In the function `summation` we abstracted the *term* to be summed

* We can go one step further and abstract also the *range* to be summed

* But the result is familiar: it is the function `sum` already defined in Python!



In [ ]:
def sum_squares(n):
  return sum(map(lambda x:x*x, range(1,n+1)))

def sum_cubes(n):
  return sum(map(lambda x:x*x*x, range(1,n+1)))

def sum_naturals(n):
  return sum(range(1, n+1))  # no need for map in this case!


### Callbacks

- In a higher-order function, when one of the parameters passed in is a function, that function is a *callback function* because it will be called back and used within the higher-order function

## 16.5 Functions as return values

### Nested functions

- A function which is defined inside another function is known as nested function

- Nested functions are able to access variables of the enclosing scope

- In Python, these non-local variables can be accessed only within their scope and not outside their scope

```
def outer_function(text):
    itext = text

    def inner_function():  # nested function
        print(itext)       # accesses 'itext' as non-local variable
  
    inner_function()

outer_function('Hey!')

```

### Scope of variables

![images](https://raw.githubusercontent.com/fp-leic/public/main/notebooks/16/scope-in-python.png)

$\Rightarrow$
https://www.datacamp.com/community/tutorials/scope-of-variables-python

In [1]:
# Think about the scopes of x and y before running this code

x = 0
y = 3
def outer():
    x = 1
    def inner():
        x = 2
        print("inner x:", x)
        print("global y:", y)

    inner()
    print("outer x:", x)

outer()
print("global x:", x)

inner x: 2
global y: 3
outer x: 1
global x: 0


$\Rightarrow$
https://www.datacamp.com/community/tutorials/scope-of-variables-python

### Closures

- A closure is representation the variables environment at the time a function was defined

-  When we return a function, Python automatically creates a closure

- For example, if we nest a function inside the encapsulating function and then return the underlying function


In [2]:
def outer(a):
    def inner(b):
        return a + b
    return inner

add5 = outer(5)
print(add5)
print(add5(10))

<function outer.<locals>.inner at 0x7ff347965c60>
15


We can achive the same result using a lambda expression:

In [3]:
def outer(a):
  return lambda b: a + b

add5 = outer(5)
print(add5)
print(add5(10))

<function outer.<locals>.<lambda> at 0x7ff347965750>
15


### Example: adding debug information to a function

Let us use nested functions to add debug information to any function.

In [4]:
def debug(fun):
  def wrapper(*args):
      print(f'calling {fun.__name__}({args})')
      result = fun(*args)
      print(f'returned {result}')
      return result
  return wrapper

def fact(n):
  f = 1
  for i in range(2,n+1):
    f = f*i
  return f

wrap_fact = debug(fact)
print(wrap_fact(5))
print(wrap_fact(10))


calling fact((5,))
returned 120
120
calling fact((10,))
returned 3628800
3628800


### Larger example: memorizing function results

Recall the "slow" recursive Fibbonacci definition:

```
def fib(n):
   if n <= 1:
       return n
   else:
       return fib(n-1) + fib(n-2)
```


Let us write a higher-order function that takes a single-argument function
and return a "memorized" version, i.e. one that records previously computed values in a dictionary.


```
def memorize(function):
    memo = dict()      # empty dictionary for results
    def wrapper(arg):
        if arg in memo:
            print(f'returning memorized argument: {arg} -> {memo[arg]}')
            return memo[arg]
        else:
            print(f'computing argument {arg} for the first time')
            result = function(arg)
            memo[arg] = result
            return result
    return wrapper   # no parenthesis
```

The second call to the memorized fib is instantaneous:

```
memofib = memorize(fib)
print(memofib(35)) # slow
print(memofib(35)) # fast, result was already computed
```

$\Rightarrow$
[https://github.com/fp-leic/public/blob/main/lectures/16/higher_order.py](https://github.com/fp-leic/public/blob/main/lectures/16/higher_order.py)



## 16.6 Utility Higher-Order Functions

### Composition

- A handy utility is `compose()` that takes a sequence of functions and returns a function that represents the application of each of these argument functions to a data argument:

In [ ]:
def compose(*funcs):
    """Return a new function compose(f,g,...)(x) == f(g(...(x)))."""
    def inner(data):
        result = data
        for f in reversed(funcs):
            result = f(result)
        return result
    return inner

In [ ]:
mod6 = lambda x: x%6
times2 = lambda x: x*2
minus3 = lambda x: x-3

f = compose(mod6, times2, minus3) # (x-3)*2)%6

In [ ]:
all(f(i)==((i-3)*2)%6 for i in range(1000))

## 16.7 Currying and partial application

### higher-order functions & currying

- We can use higher-order functions to convert a function that takes multiple arguments into a chain of functions that each take a single argument

- More specifically, given a function `f(x, y)`, we can define a function `g` such that `g(x)(y)` is equivalent to `f(x, y)`

- Here, `g` is a higher-order function that takes in a single argument `x` and returns another function that takes in a single argument `y`

- This transformation is called *currying*<sup>1</sup>

<sup>1</sup> Named after [Haskell Curry](https://en.wikipedia.org/wiki/Haskell_Curry)

- As an example, we can define a curried version of the `pow()` function:

In [5]:
def curried_pow(x):
    def inner(y):
        return pow(x, y)
    return inner

curried_pow(2)(3)

8

- *Currying* is useful when we require a function that takes in only a single argument

- For example, the map pattern applies a single-argument function to a sequence of values

In [ ]:
list(map(curried_pow(2), range(10)))

## 16.8 The `functools` Module

### The `functools` Module HOF

- The `functools` module contains some higher-order functions

  - `partial()`, `reduce()`

  - https://docs.python.org/3/library/functools.html

- The `operator` modules defines function that behave identically to Python operators (e.g. +, *, etc.)

- we already saw `reduce()`

```
>>> import functools, operator
>>> functools.reduce(operator.add, [1, 2, 3, 4], 10)
20
>>> sum([1, 2, 3, 4])
10
```

  

### Partial function application

- The most useful tool in this module is the `functools.partial()` function

- For programs written in a functional style, you’ll sometimes want to construct variants of existing functions that have some of the parameters filled in

- Consider a Python function `f(a, b, c)`; you may wish to create a new function `g(b, c)` that’s equivalent to `f(1, b, c)`; you’re filling in a value for one of f()’s parameters

- This is called “partial function application”

- Currying is often confused with partial application

> Where partial application takes a function and from it builds a function which takes fewer arguments, currying builds functions which take multiple arguments by composition of functions which each take a single argument.

$\Rightarrow$
https://en.wikipedia.org/wiki/Partial_application

```
import functools

def log(message, subsystem):
    """Write the contents of 'message' to the specified subsystem."""
    print('%s: %s' % (subsystem, message))
    ...

server_log = functools.partial(log, subsystem='server')
server_log('Unable to open socket')
```
$\Rightarrow$
https://docs.python.org/3/howto/functional.html#the-functools-module

In [6]:
from functools import partial
basetwo = partial(int, base=2)
basetwo.__doc__ = 'Convert base 2 string to an int.'
basetwo('10010')

18

## 16.9 The `operator` Module

### Useful Function Objects

- There are many builtin functions in Python that accept functions as arguments

- An example is the `filter()` function that was used previously

- However, there are some basic actions that use operators instead of functions (like `+` or the subscript `[]` or dot `.` operators)

- The `operator` module provides function versions of these operators

```
>>> import operator
>>> operator.add(1, 2)
3
```

$\Rightarrow$
https://docs.python.org/3/library/operator.html

Operator modules operations may replace the use of many lambda functions; for example `operator.concat`:

In [ ]:
import operator, functools
functools.reduce(operator.concat, ['A', 'BB', 'A'])

Operation `operator.itemgetter()` may also be used as key to sort collections:

In [7]:
ledzep = [('Guitar', 'Jimmy'), ('Vocals', 'Robert'), ('Bass', 'John Paul'), ('Drums', 'John')]

sorted(ledzep)

[('Bass', 'John Paul'),
 ('Drums', 'John'),
 ('Guitar', 'Jimmy'),
 ('Vocals', 'Robert')]

In [8]:
import operator

sorted(ledzep, key=operator.itemgetter(1))

[('Guitar', 'Jimmy'),
 ('Drums', 'John'),
 ('Bass', 'John Paul'),
 ('Vocals', 'Robert')]

$\Rightarrow$
https://www.protechtraining.com/content/python_fundamentals_tutorial-functional_programming

# Further reading

### Function decorators (bonus)

- Yes, there's more....

$\Rightarrow$
[Primer on Python Decorators](https://realpython.com/primer-on-python-decorators/#simple-decorators)

$\Rightarrow$
[13.10. Decorators](https://www.protechtraining.com/content/python_fundamentals_tutorial-functional_programming)


### Lambda Expressions & Anonymous Functions

Python Tutorial || Learn Python Programming -- Socratica

In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo('25ovCm9jKfA')

-- Nuno Macedo, João Correia Lopes & Pedro Vasconcelos